In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA,SparsePCA
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from scipy.stats import uniform, truncnorm, randint
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.utils import class_weight
import gensim
from gensim.models import KeyedVectors, Word2Vec, FastText
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,classification_report
import seaborn as sns

In [2]:
X_link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/X.csv'
X = pd.read_csv(X_link, header = None).to_numpy()
label_link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/label.csv'
label = pd.read_csv(label_link, header = None)
label = label.to_numpy().reshape([1, len(label)])[0]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,label, test_size=0.2)

In [4]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[0. 1.], y=[0. 1. 1. ... 0. 1. 1.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [5]:
class_weights


array([1.46770375, 0.75834353])

In [6]:
cw = {0:2.2*class_weights[0],1:class_weights[1]}
cw_DT = {0:class_weights[0],1:class_weights[1]}

In [7]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [58,60,62,64],
    'criterion' : ['gini', 'entropy']
}

In [22]:
rfc=RandomForestClassifier(class_weight = cw)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid)
CV_rfc.fit(X_train, y_train)
y_pred = CV_rfc.predict(X_test)
CV_rfc.best_params_

{'criterion': 'entropy', 'max_depth': 62, 'max_features': 'sqrt'}

In [23]:
CV_rfc.best_score_

0.7165898617511521

In [24]:
print("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
print("Precision :",metrics.precision_score(y_test,y_pred))
print("F1-Score  :",metrics.f1_score(y_test,y_pred))
print("Recall    :",metrics.recall_score(y_test,y_pred))

Accuracy  : 0.7171810225702441
Precision : 0.7138381201044386
F1-Score  : 0.8166069295101552
Recall    : 0.9539427773900907


In [25]:
print(confusion_matrix(y_test,y_pred))

[[ 190  548]
 [  66 1367]]


In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.74      0.26      0.38       738
         1.0       0.71      0.95      0.82      1433

    accuracy                           0.72      2171
   macro avg       0.73      0.61      0.60      2171
weighted avg       0.72      0.72      0.67      2171



In [27]:
CV_rfc.best_params_

{'criterion': 'entropy', 'max_depth': 62, 'max_features': 'sqrt'}

In [9]:
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': list(range(2, 10)),'criterion' :['gini', 'entropy'],'max_depth':list(range(2, 10))}
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(class_weight = cw_DT), params)
grid_search_cv.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(class_weight={0: 1.4677037538045317,
                                                            1: 0.758343526122663}),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9]})

In [10]:
y_pred=grid_search_cv.predict(X_test)

In [11]:
print("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
print("Precision :",metrics.precision_score(y_test,y_pred))
print("F1-Score  :",metrics.f1_score(y_test,y_pred))
print("Recall    :",metrics.recall_score(y_test,y_pred))

Accuracy  : 0.6047904191616766
Precision : 0.7167859984089101
F1-Score  : 0.6774436090225564
Recall    : 0.6421952957947256


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.45      0.54      0.49       768
         1.0       0.72      0.64      0.68      1403

    accuracy                           0.60      2171
   macro avg       0.58      0.59      0.58      2171
weighted avg       0.62      0.60      0.61      2171



In [13]:
confusion_matrix(y_test,y_pred)

array([[412, 356],
       [502, 901]])

In [14]:
 grid_search_cv.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_leaf_nodes': 8,
 'min_samples_split': 2}

In [1]:
cf_matrix = np.array([[145, 632], [113,1281]])

NameError: name 'np' is not defined

In [2]:
group_names = ['True Pos','False Pos','False Neg','True Neg']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'\n{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

NameError: name 'cf_matrix' is not defined